In [1]:
import pyvista as pv
import numpy as np
import itertools

In [2]:
fluid = pv.read("../data/raw/pressure_analysis/z4/VTKFluid/Fluid_t200000.vtr").point_data_to_cell_data()

In [8]:
fluid.cell_data

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : obstacles
Active Vectors  : velocityVector
Active Texture  : None
Active Normals  : None
Contains arrays :
    inertialStressTensor    float32    (7185591, 9)
    obstacleDistance        float32    (7185591,)
    obstacles               float32    (7185591,)           SCALARS
    pressure                float32    (7185591,)
    velocityVector          float32    (7185591, 3)         VECTORS
    viscousStressTensor     float32    (7185591, 9)         TENSORS

In [11]:
mesh = pv.read("../data/raw/pressure_analysis/z4/VTKParticles/Particles_t200000.vtp")
mesh

PolyData (0x7f1312074640)
  N Cells:    7220
  N Points:   3612
  N Strips:   0
  X Bounds:   4.763e+01, 7.965e+01
  Y Bounds:   6.498e+01, 9.702e+01
  Z Bounds:   3.728e+01, 6.926e+01
  N Arrays:   4

In [15]:
results = mesh.sample(fluid)
results.cell_data

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : areaDeviation
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    areaDeviation           float32    (7220,)              SCALARS
    maxAngle                float32    (7220,)
    particleIndex           int32      (7220,)
    rank                    int32      (7220,)
    vtkGhostType            uint8      (7220,)

In [17]:
results.point_normals

pyvista_ndarray([[-0.7496653 , -0.53576785,  0.38852897],
                 [ 0.7622213 , -0.54049253, -0.35621136],
                 [-0.7496653 ,  0.53576785,  0.38852897],
                 ...,
                 [ 0.05388154,  0.11166087,  0.9922846 ],
                 [ 0.09392958,  0.05648666,  0.9939751 ],
                 [ 0.03473065,  0.05592673,  0.9978306 ]], dtype=float32)

In [18]:
results.points

pyvista_ndarray([[75.81499 , 89.46909 , 46.689808],
                 [51.27837 , 89.477615, 59.90226 ],
                 [75.81499 , 72.53091 , 46.689808],
                 ...,
                 [63.298992, 79.20905 , 37.405758],
                 [62.659184, 80.09349 , 37.3785  ],
                 [63.605587, 80.10281 , 37.316803]], dtype=float32)

In [19]:
results.is_all_triangles

True

In [20]:
results.is_manifold

True

In [21]:
results.faces

array([   3,   30,   12, ...,  534, 3611,  210])

In [22]:
results.volume

17131.502029781353

In [23]:
results.verts

array([], dtype=int64)

In [24]:
test = results.compute_cell_sizes(length=False, area=True, volume=False)

In [25]:
test

PolyData (0x7f130773d3c0)
  N Cells:    7220
  N Points:   3612
  N Strips:   0
  X Bounds:   4.763e+01, 7.965e+01
  Y Bounds:   6.498e+01, 9.702e+01
  Z Bounds:   3.728e+01, 6.926e+01
  N Arrays:   14

In [26]:
test.cell_data["Area"].sum()

3216.463861207664

In [27]:
test = test.cell_data_to_point_data()

In [28]:
test

PolyData (0x7f130773d900)
  N Cells:    7220
  N Points:   3612
  N Strips:   0
  X Bounds:   4.763e+01, 7.965e+01
  Y Bounds:   6.498e+01, 9.702e+01
  Z Bounds:   3.728e+01, 6.926e+01
  N Arrays:   14

In [29]:
test.point_data["Area"].sum()


1609.0589386284582

In [30]:
test.n_cells

7220

In [31]:
test.n_faces

7220

In [32]:
test.n_points

3612

In [33]:
test.n_lines

0

In [34]:
test.points

pyvista_ndarray([[75.81499 , 89.46909 , 46.689808],
                 [51.27837 , 89.477615, 59.90226 ],
                 [75.81499 , 72.53091 , 46.689808],
                 ...,
                 [63.298992, 79.20905 , 37.405758],
                 [62.659184, 80.09349 , 37.3785  ],
                 [63.605587, 80.10281 , 37.316803]], dtype=float32)

In [35]:
weights = np.zeros(test.n_points)
for cell in test.cell:
    vec_a = cell.points[1] - cell.points[0]
    vec_b = cell.points[2] - cell.points[0]
    area = 0.5 * np.linalg.norm(np.cross(vec_a, vec_b))
    for vert in cell.point_ids:
        weights[vert] += area / 3.0
test.point_data["weights"] = weights

In [36]:
weights

array([0.69105104, 0.68894765, 0.69105104, ..., 0.85716496, 0.85692035,
       0.84711381])

In [44]:
traction = np.empty((0,3))
devForce = np.empty((0,3))
inertialForce = np.empty((0,3))
for stress, normal, weight, press in zip(test.point_data["viscousStressTensor"], test.point_normals, test.point_data["weights"], test.point_data["pressure"]):
    viscousStress = stress.reshape((3,3))
    pressTensor = np.identity(3) * press
    tractionTensor = viscousStress + pressTensor
    traction = np.vstack((traction, tractionTensor @ normal * weight))    
    devForce = np.vstack((devForce, viscousStress @ normal * weight))
    inertialForce = np.vstack((inertialForce, pressTensor @ normal * weight))

test.point_data["traction"] = traction
test.point_data["devForce"] = devForce
test.point_data["inertialForce"] = inertialForce

test.save('test.vtp')

array([[-7.48106107e-06,  6.59105158e-06,  1.72612069e-05],
       [-3.07727774e-05,  3.03773234e-05, -2.13609689e-05],
       [-7.48106107e-06, -6.59105158e-06,  1.72612069e-05],
       ...,
       [ 1.24698315e-05,  2.01793599e-06,  1.39038503e-05],
       [ 1.31064802e-05,  1.10088954e-06,  1.57256274e-05],
       [ 1.18572941e-05,  9.70669760e-07,  1.60652299e-05]])

In [38]:
test.point_data["pressure"]

pyvista_ndarray([ 1.3874999e-05, -4.7875001e-05,  1.3874999e-05, ...,
                  1.7956250e-05,  1.9428750e-05,  1.9261250e-05],
                dtype=float32)

In [39]:
test.point_normals

pyvista_ndarray([[-0.7496653 , -0.53576785,  0.38852897],
                 [ 0.7622213 , -0.54049253, -0.35621136],
                 [-0.7496653 ,  0.53576785,  0.38852897],
                 ...,
                 [ 0.05388154,  0.11166087,  0.9922846 ],
                 [ 0.09392958,  0.05648666,  0.9939751 ],
                 [ 0.03473065,  0.05592673,  0.9978306 ]], dtype=float32)

In [40]:
np.zeros((test.n_points, 3))

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [41]:
weights = np.zeros(test.n_points)
for cell in test.cell:
    vec_a = cell.points[1] - cell.points[0]
    vec_b = cell.points[2] - cell.points[0]
    area = 0.5 * np.linalg.norm(np.cross(vec_a, vec_b))
    for vert in cell.point_ids:
        weights[vert] += area / 3.0
test.point_data["weights"] = weights